In [0]:
!pip install dotenv

### Setup

In [0]:
# Budget Setup and Management Notebook
# This notebook helps create, list, and retire budget policies with appropriate tags and permissions

# Import necessary libraries
import os
import requests
from dotenv import load_dotenv
import json
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets

# Load environment variables
load_dotenv()

# Configuration parameters
ACCOUNT_HOST = os.getenv("ACCOUNT_HOST", "https://accounts.cloud.databricks.com")
ACCOUNT_ID = os.getenv("ACCOUNT_ID", "")
CLIENT_ID = os.getenv("CLIENT_ID", "")
CLIENT_SECRET = os.getenv("CLIENT_SECRET", "")
# Workspace-level API authentication settings
TOKEN = os.getenv("TOKEN")  # PAT token
DATABRICKS_INSTANCE = os.getenv("DATABRICKS_INSTANCE")  # e.g., "adb-1234567890123456.7.azuredatabricks.net"

# Maximum number of tag pairs to support
MAX_TAGS = 3

### Authentication Methods

In [0]:
# Budget Setup and Management Notebook
# Authentication for both account-level and workspace-level APIs

# Account-level authentication function
def get_oauth_token():
    """Get OAuth token for account-level APIs"""
    token_url = f"https://accounts.azuredatabricks.net/oidc/accounts/{ACCOUNT_ID}/v1/token"
    
    response = requests.post(
        token_url,
        auth=(CLIENT_ID, CLIENT_SECRET),
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data="grant_type=client_credentials&scope=all-apis"
    )
    
    if response.status_code == 200:
        token_data = response.json()
        return token_data["access_token"]
    else:
        print(f"Error getting token: {response.status_code}")
        print(response.text)
        return None

# Test account-level authentication
def test_account_auth():
    token = get_oauth_token()
    
    if not token:
        print("Failed to obtain OAuth token.")
        return False
    
    url = f"https://accounts.azuredatabricks.net/api/2.1/accounts/{ACCOUNT_ID}/budget-policies"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        print("✅ Account authentication successful!")
        return True
    else:
        print(f"❌ Account authentication failed: {response.status_code}")
        print(response.text)
        return False

# Test workspace-level authentication
def test_workspace_auth():
    if not TOKEN or not DATABRICKS_INSTANCE:
        print("❌ TOKEN or DATABRICKS_INSTANCE not set.")
        return False
    
    url = f"https://{DATABRICKS_INSTANCE}/api/2.0/policies/clusters/list"
    headers = {
        "Authorization": f"Bearer {TOKEN}",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            print("✅ Workspace authentication successful!")
            return True
        else:
            print(f"❌ Workspace authentication failed: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Error testing workspace authentication: {str(e)}")
        return False

# Test both authentication methods
account_auth = test_account_auth()
workspace_auth = test_workspace_auth()

# Store the account token if successful
if account_auth:
    ACCOUNT_TOKEN = get_oauth_token()
    print(f"Account OAuth Token obtained: {ACCOUNT_TOKEN[:10]}...{ACCOUNT_TOKEN[-10:]} (truncated)")
else:
    ACCOUNT_TOKEN = None
    print("No account token available")

print(f"Authentication status: Account={account_auth}, Workspace={workspace_auth}")

### Budget Policy Management

In [0]:
# Budget Policy Management Functions

def create_budget_policy(budget_name, custom_tags):
    """
    Creates a budget policy with specified name and tags
    
    Args:
        budget_name (str): Name of the budget policy
        custom_tags (list): List of dicts with key/value pairs
    
    Returns:
        str: The policy ID of the created budget policy
    """
    url = f"https://accounts.azuredatabricks.net/api/2.1/accounts/{ACCOUNT_ID}/budget-policies"
    
    # Prepare the budget policy request
    policy_request = {
        "policy": {
            "policy_name": budget_name,
            "custom_tags": custom_tags
        }
    }
    
    headers = {
        "Authorization": f"Bearer {ACCOUNT_TOKEN}",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.post(
            url,
            headers=headers,
            data=json.dumps(policy_request)
        )
        
        if response.status_code == 200:
            result = response.json()
            policy_id = result.get("policy_id")
            print(f"✅ Budget policy created successfully. Policy ID: {policy_id}")
            return policy_id
        else:
            print(f"❌ Error creating budget policy: {response.status_code}")
            print(response.text)
            return None
    except Exception as e:
        print(f"❌ Exception creating budget policy: {e}")
        return None

def list_budget_policies():
    """
    Lists all budget policies in the account
    
    Returns:
        list: All budget policies
    """
    url = f"https://accounts.azuredatabricks.net/api/2.1/accounts/{ACCOUNT_ID}/budget-policies"
    
    headers = {
        "Authorization": f"Bearer {ACCOUNT_TOKEN}",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            result = response.json()
            policies = result.get("policies", [])
            
            # Display the policies in a table format
            import pandas as pd
            
            if policies:
                policy_data = []
                for policy in policies:
                    policy_dict = {
                        "Policy ID": policy.get("policy_id", "N/A"),
                        "Policy Name": policy.get("policy_name", "N/A"),
                        "Tags": ", ".join([f"{tag.get('key')}:{tag.get('value')}" 
                                         for tag in policy.get("custom_tags", [])])
                    }
                    policy_data.append(policy_dict)
                
                df = pd.DataFrame(policy_data)
                display(HTML(df.to_html(index=False)))
            else:
                print("No budget policies found.")
            
            return policies
        else:
            print(f"❌ Error listing budget policies: {response.status_code}")
            print(response.text)
            return []
    except Exception as e:
        print(f"❌ Exception listing budget policies: {e}")
        return []

def retire_budget_policy(policy_id):
    """
    Retires (deletes) a budget policy
    
    Args:
        policy_id (str): The ID of the policy to retire
    
    Returns:
        bool: Success or failure
    """
    url = f"https://accounts.azuredatabricks.net/api/2.1/accounts/{ACCOUNT_ID}/budget-policies/{policy_id}"
    
    headers = {
        "Authorization": f"Bearer {ACCOUNT_TOKEN}",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.delete(url, headers=headers)
        
        if response.status_code == 200:
            print(f"✅ Budget policy {policy_id} successfully retired.")
            return True
        else:
            print(f"❌ Error retiring budget policy: {response.status_code}")
            print(response.text)
            return False
    except Exception as e:
        print(f"❌ Exception retiring budget policy: {e}")
        return False

### Interaction and execution

In [0]:
# UI Interactions and Main Execution Logic

# Create widgets for budget definition
budget_name = widgets.Text(
    description='Budget Name:',
    placeholder='Enter budget name',
    style={'description_width': 'initial'}
)

# Custom tag widgets - allow for 3 key-value pairs
tag_widgets = []
for i in range(1, 4):
    key = widgets.Text(
        description=f'Tag Key {i}:',
        placeholder='Enter key',
        style={'description_width': 'initial'}
    )
    value = widgets.Text(
        description=f'Tag Value {i}:',
        placeholder='Enter value',
        style={'description_width': 'initial'}
    )
    tag_widgets.extend([key, value])

# Retirement widget
policy_id_to_retire = widgets.Text(
    description='Policy ID to Retire:',
    placeholder='Enter policy ID',
    style={'description_width': 'initial'}
)

# Action dropdown
action = widgets.Dropdown(
    options=['None', 'Create Budget', 'List Budgets', 'Retire Budget'],
    value='None',
    description='Action:',
    style={'description_width': 'initial'}
)

# Button for execution
execute_button = widgets.Button(
    description='Execute Action',
    button_style='primary',
    icon='play'
)

# Status output
status_output = widgets.Output()

# Function to execute when button is clicked
def on_execute_button_clicked(b):
    with status_output:
        status_output.clear_output()
        print("Running action...")
        
        selected_action = action.value
        
        if selected_action == "Create Budget":
            # Collect input values
            name = budget_name.value
            
            # Validate inputs
            if not name:
                print("❌ Budget name is required.")
                return
            
            # Collect custom tags
            custom_tags = []
            for i in range(0, len(tag_widgets), 2):
                key = tag_widgets[i].value
                value = tag_widgets[i+1].value
                if key and value:
                    custom_tags.append({"key": key, "value": value})
            
            if not custom_tags:
                print("⚠️ Warning: No tags specified. Budget will be created without tags.")
            
            # Create budget policy
            policy_id = create_budget_policy(name, custom_tags)
            
        elif selected_action == "List Budgets":
            list_budget_policies()
        
        elif selected_action == "Retire Budget":
            policy_id = policy_id_to_retire.value
            if policy_id:
                retire_budget_policy(policy_id)
            else:
                print("❌ Please enter a policy ID to retire.")
        
        elif selected_action == "None":
            print("Please select an action from the dropdown")

# Connect the button click event to the handler function
execute_button.on_click(on_execute_button_clicked)

# Display the widgets
display(Markdown("## Budget Policy Management"))
display(budget_name)
display(Markdown("### Custom Tags"))
for widget in tag_widgets:
    display(widget)
display(Markdown("### Budget Policy Actions"))
display(action)
display(policy_id_to_retire)
display(execute_button)
display(status_output)